In [1]:
import sys, getopt
import pandas as pd
import numpy as np
import os

digi is a company that sells reward -- amazing :)

### Read Data

In [2]:
# inputfile = '/Users/ito/Downloads/AppAprMonthlyRewards.csv'
# inputfile = '/Users/vidyut/projects/digi/yiteng_code/180625/AppAprMonthlyRewards.csv'
inputfile = 'MonthlyRewards.xlsx'

In [3]:
# details = pd.read_csv(inputfile, sep=',', engine='c') # , parse_dates=['TimeStamp']
df_collection = [pd.read_excel('MonthlyRewards.xlsx', sheetname=idx) for idx in range(5) ]
# print details.shape

/Users/ito/venv/pyspark-rec/lib/python2.7/site-packages/pandas/util/_decorators.py:118: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  return func(*args, **kwargs)


### Data Cleansing

* the first row is depicted with description for each feature/attribute
* hence cut the first one, and select some useful features to analyse
* exp-based Feature Selection

In [4]:
useful_features = ['Category','VoucherName','Length','MerchantName','RedemptionType','Category1',
          'VoucherType','NoCodes','Description','TermsConditions']

def df_cleasing(sub_df):
    sub_df = sub_df[1:]
    sub_df = sub_df[['Category','Remarks','Group Name/Voucher Name','Length','Merchant Name (Tag)',
                   'Redemption Type  (Tag)','Category.1','Voucher Type','Require daily cap?','Daily Cap Quantity',
                   'Require per user limit?','Per user limit Quantity','Total # of Codes','Campaign Start Date',
                   'Campaign End Date','Voucher Expiry Date','Promo Code','Description in Goods',
                   'Terms and Conditions in Goods']]
    sub_df.columns = ['Category','Remarks','VoucherName','Length','MerchantName','RedemptionType','Category1',
          'VoucherType','RequireDailyCap','DailyCapQuantity','RequireUserLimit','UserLimitQuantity','NoCodes',
          'CampaignStartDate','CampaignEndDate','VoucherExpiryDate','PromoCode','Description','TermsConditions']
    sub_df = sub_df[useful_features]
    return sub_df

* Clean N.A. NaN

In [5]:
details = pd.concat([df_cleasing(sub_df) for sub_df in df_collection])
print details.shape
details = details.dropna(axis=0, thresh=10)  
print details.shape

(848, 10)
(662, 10)


In [6]:
details.head(3)

,Category,VoucherName,Length,MerchantName,RedemptionType,Category1,VoucherType,NoCodes,Description,TermsConditions
1,Monthly Rewards,RebateMango: Up to FREE RM100 cashback,38.0,RebateMango,online,CAT=SHOPPING,embedded promo code\n,9999,MyDigi Reward:\nInstantly get up to FREE RM100...,Terms and Conditions:\nValid from now till 31 ...
2,Monthly Rewards,Agoda: Additional 8% off in January,35.0,Agoda,online,CAT=TRAVEL,unique promo code\n,10000,MyDigi Reward:\nAdditional 8% off on your booking,Terms and Conditions:\nValid from now till 31 ...
3,Monthly Rewards,Agoda: Additional 10% off in January,36.0,Agoda,online,CAT=TRAVEL,embedded promo code\n,10000,MyDigi Reward:\nAdditional 10% off on your boo...,Terms and Conditions:\nValid from now till 31 ...


In [7]:
type(details.Category1.iloc[0])

unicode

* a preliminary process -- transferring all the chars to lowercase

* for later when we process the data, stemming should be considered as well (not that helpful at this stage)

In [8]:
def lowercase(input):
    if type(input) == str:
        return input.decode('utf-8').lower()
    elif type(input) == unicode:
        return input.lower()
    else:
        return input

In [9]:
details = details.replace(np.nan, 'None')
for each in details:
    details[each] = details[each].map(lowercase)

### Understanding the Voucher Name

* split by a colon (may need to modify in case the format changes)

In [10]:
def voucher_name_split(input):
    sub_strs = input.split(': ')
    if len(sub_strs)!=2:
        sys.stdout.errors('new case...')
    try:
        return sub_strs[0], sub_strs[1]
    except:
        sys.stdout.errors('lesser than 2...')
        
res_split = details.VoucherName.map(voucher_name_split)
details[['location', 'voucher']] = res_split.apply(lambda x: pd.Series([i for i in x]))

In [11]:
details.Category1.unique()

array([u'cat=shopping', u'cat=travel', u'cat=f&b', u'cat=others',
       u'cat=beauty', u'cat=digital'], dtype=object)

In [12]:
def category_split(input):
    sub_strs = input.split('cat=')
    if len(sub_strs)!=2:
        sys.stdout.errors('new case...')
    try:
        return sub_strs[1]
    except:
        sys.stdout.errors('unknown situation')
        
details['hierarchy'] = details.Category1.map(category_split)

* based on the content, generate a "keyword dict" to extract the type of reward

In [13]:
word_lst = [['buy','free'], ['buy','for the price'], '% discount','discount','free gift','free', 
            'cash voucher','gift voucher', '% off', 'off', 'complimentary', 
            'instant rebate', 'rebate','cashback','x reward'] 

# value_lst = [?, ?, per, real, ]
# word_lst2 = ['up to', 'extra', '']

In [14]:
def list_check(lst_ele, voucher):
    if lst_ele in voucher:
        return 0
    else:
        return 1

In [15]:
def categorize(each_voucher):
    for idx in range(details.shape[0]):
        category = None
        for which, each in enumerate(word_lst):
            if type(each) == list:
                if sum([list_check(ele, each_voucher) for ele in each]) == 0:
                    category = which
                    break
            elif each in each_voucher:
                category = which
                break
        if (category == None) & ('rm' in each_voucher):
            # trick
            category = len(word_lst)

        return category
details['voucher_cat'] = details.voucher.map(categorize)
details['voucher_cat'] = details.voucher_cat.astype(int)

In [16]:
details.head(3)

,Category,VoucherName,Length,MerchantName,RedemptionType,Category1,VoucherType,NoCodes,Description,TermsConditions,location,voucher,hierarchy,voucher_cat
1,monthly rewards,rebatemango: up to free rm100 cashback,38.0,rebatemango,online,cat=shopping,embedded promo code\n,9999,mydigi reward:\ninstantly get up to free rm100...,terms and conditions:\nvalid from now till 31 ...,rebatemango,up to free rm100 cashback,shopping,5
2,monthly rewards,agoda: additional 8% off in january,35.0,agoda,online,cat=travel,unique promo code\n,10000,mydigi reward:\nadditional 8% off on your booking,terms and conditions:\nvalid from now till 31 ...,agoda,additional 8% off in january,travel,8
3,monthly rewards,agoda: additional 10% off in january,36.0,agoda,online,cat=travel,embedded promo code\n,10000,mydigi reward:\nadditional 10% off on your boo...,terms and conditions:\nvalid from now till 31 ...,agoda,additional 10% off in january,travel,8


### Feature Eng

* For this, I will hence calculate a function that representing the voucher value (will do after NLP)

* details.RedemptionType.unique() -- array([u'offline', u'online'], dtype=object)
* details.VoucherType.unique() -- array([u'generic promo code\n', u'embedded promo code\n', u'unique promo code\n'], dtype=object)

In [17]:
RedemType_dic, VouchType_dic = dict(), dict()
RedemType_dic.update({'offline':0, 'online':1})
VouchType_dic.update({'generic promo code\n':1, 'embedded promo code\n':2, 'unique promo code\n':3})
details['RedemptionType'] = details.RedemptionType.map(RedemType_dic)
details['VoucherType'] = details.VoucherType.map(VouchType_dic)

* From number of codes, we can extract the value of the reward/voucher, using 1mil score as a base

* This is the current way of using NoCodes, should check whether this is correct / balanced
* Current approach may be wrong

In [18]:
# details.Length.unique()
details.NoCodes.unique()

array([9999, 10000, 30000, 99999, 1800, 3000, 500000, 347, 200, 11000,
       25000, 100000, 5000, u'unlimited', 800, 2000, 14000, 187072, 186976,
       200000, 199895, 1000, 3001, 100, 4000, 400, 500], dtype=object)

In [19]:
# NoCodes_dic = dict()
# NoCodes_dic.update({'1,800':555.56, '9,999':100.01, '500,000':2.0, '99,999':10.0, 'unlimited':1.0, '2,000':500.0,
#        '3,000':333.33, '10,000':100, '1,000':1000, '3,001':333.2, '30,000':33.33, '200':5000, '100':10000})
# details['value'] = details.NoCodes.map(NoCodes_dic)

In [20]:
details.hierarchy.unique()

array([u'shopping', u'travel', u'f&b', u'others', u'beauty', u'digital'], dtype=object)

In [21]:
# hierarchy_dic = dict()
# hierarchy_dic.update({'shopping':1 , 'beauty':2 , 'others':3 , 'f&b':4 , 'travel':5 , 'digital':6})
# details['hierarchy'] = details.hierarchy.map(hierarchy_dic)

In [22]:
details.head(2)

,Category,VoucherName,Length,MerchantName,RedemptionType,Category1,VoucherType,NoCodes,Description,TermsConditions,location,voucher,hierarchy,voucher_cat
1,monthly rewards,rebatemango: up to free rm100 cashback,38.0,rebatemango,1,cat=shopping,2,9999,mydigi reward:\ninstantly get up to free rm100...,terms and conditions:\nvalid from now till 31 ...,rebatemango,up to free rm100 cashback,shopping,5
2,monthly rewards,agoda: additional 8% off in january,35.0,agoda,1,cat=travel,3,10000,mydigi reward:\nadditional 8% off on your booking,terms and conditions:\nvalid from now till 31 ...,agoda,additional 8% off in january,travel,8


### Selection for clustering

* Category seems not usable in this case, as there is no diversity in this feature
* VoucherName is hence considered in the tf-idf model
* Length is just, the length of VoucherName...
* MerchantName is a double check feature for first seg of VoucherName
* NoCodes is hence translated to value
* voucher_cat is designed based on 
* hierarchy is categorized -- {'shopping', 'beauty', 'others', 'f&b', 'travel', 'digital'}

In [23]:
clu_feas = ['RedemptionType', 'VoucherType', 'voucher_cat', 'hierarchy']
sub_details = details[clu_feas]

### 1. Text Features Eng, according to current tf-idf

In [24]:
def description_split(input):
    sub_strs = input.split(':\n')
    if len(sub_strs)!=2:
        sys.stdout.errors('new case...')
    try:
        return sub_strs[1]
    except:
        sys.stdout.errors('lesser than 2...')
        
details['desc'] = details.Description.map(description_split)

In [25]:
def description_split(input):
    if 'conditions:\n' in input:
        sub_strs = input.split('conditions:\n')
    elif 'details:\n' in input:
        sub_strs = input.split('details:\n')
        
    if len(sub_strs)!=2:
        print sub_strs
        sys.stdout.errors('new case...')
    try:
        return sub_strs[1]
    except:
        sys.stdout.errors('lesser than 2...')
details['tnc'] = details.TermsConditions.map(description_split)

### NLP task on "reward similarity consideration"

In [26]:
import sklearn

from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer  

from sklearn.metrics.pairwise import linear_kernel

### stop word consideration 

* This transfers words to a word-frequency matrix (initialization)
* Add analyzer, stop words and tokenization as future work

In [27]:
vectorizer=CountVectorizer(analyzer='word', stop_words='english', token_pattern='\\b\\w+\\b')

* This counts the tf-idf weight for each word (initialization) 

In [28]:
transformer=TfidfTransformer()

* fit to our data

### Text Feature Extraction Module

#### For all categories considered

In [29]:
def tfidf_analysis(dataframe, xxx):

    tfidf=transformer.fit_transform(vectorizer.fit_transform(dataframe[xxx].values))

    # * get all words from the bag of words model

    word = vectorizer.get_feature_names()
    # for w in word:
    #     print w

    #### * extract tf-idf matrix, where the element a[i][j] represents the word_j's tf-idf-weight in text_i (each row)

#     # showcase
#     weight=tfidf.toarray()
#     for i in range(len(weight)):
#         print "-------print out all tf-idf weights for text_" + str(i) + "-------"
#         for j in range(len(word)):
#             if weight[i][j]>0:
#                 print word[j],weight[i][j]  

    doc=0
    feature_index = tfidf[doc,:].nonzero()[1]

    tfidf_scores = zip(feature_index,[word[x] for x in feature_index], [tfidf[doc, x] for x in feature_index])
    return tfidf, tfidf_scores   

In [30]:
def words_similarity(tfidf):
    arr_all_cos_sim = np.empty((0,), float)
    cosine_similarities = linear_kernel(tfidf[0:1], tfidf).flatten()
    for doc_num in range(tfidf.shape[0]):
    #     use numpy's linear kernel similarity metrics to determine similarity of each reward / document to all others
    #     flatten the result to get a 1-D array for the reward 
        all_cos_sim = linear_kernel(tfidf[doc_num:doc_num+1,], tfidf).flatten()
    #     add the similarity array for the current document, to the overall 2D similarity matrix (shape = # of docs * # of docs) 
        arr_all_cos_sim = np.append(arr_all_cos_sim, all_cos_sim, axis=0)
    arr_all = arr_all_cos_sim.reshape(tfidf.shape[0], tfidf.shape[0])
    return arr_all

### 2. Segmentation 

In [44]:
details.hierarchy.unique()

array([u'shopping', u'travel', u'f&b', u'others', u'beauty', u'digital'], dtype=object)

In [31]:
cats = ['shopping', 'beauty', 'others', 'f&b', 'travel', 'digital']

regarding the number of samples, here the PoC is with 'shopping', 'beauty', 'f&b'

In [32]:
details_shopping = details[details.hierarchy == 'shopping']
details_beauty = details[details.hierarchy == 'beauty']
details_fnb = details[details.hierarchy == 'f&b']
print details_shopping.shape

(220, 16)


### Result Organization Module

#### Exp-based, should have updated according to the rewards listed

* The rules below are generated according to the real sense of the key words from the vouchers
* This function will return a score in [0, top_n-1], indicating "accuracy" in this case
* subfunction

In [33]:
# 0==1 2==3==8==9 4==5==10 6==7 11==12
# top_n stands for the number that with relatively higher similarity
top_n = 6

def judge_rule(voucher_window):
    tolerance = [15]
    base = voucher_window.iloc[0].voucher_cat
    if base==0|base==1:
        tolerance = [0,1]
    elif base==2|base==3|base==8|base==9:
        tolerance = [2,3,8,9]
    elif base==4|base==5|base==10:
        tolerance = [4,5,10]
    elif base==7|base==6:
        tolerance = [6,7]
    elif base==11|base==12|base==13:
        tolerance = [11,12,13]
    elif base==14:
        tolerance = [14]
    elif base==15:
        tolerance = [15]
    score = [1 if voucher_window.iloc[idx].voucher_cat else 0 in tolerance for idx in range(1,top_n)]
    return sum(score)

* Recall the data to extract the corresponding content from the dataframe for final result
* Main function that involves "judge_rule(voucher_window)"

In [34]:
def similar_reward_extraction(indice_mat):
# for each reward
    final_similar_desc = []  # a container for the result
    for rw in range(indice_mat.shape[0]):
        top_rw_idx = list(indice_mat[rw][-top_n:])
        top_rw_idx.reverse()
        voucher_window = details.iloc[top_rw_idx]
        score = []
        score.append(judge_rule(voucher_window))
        doc_desc = list(voucher_window.desc)
        doc_name = list(voucher_window.VoucherName)
        tmp_doc_both = doc_desc + doc_name + score
        final_similar_desc.append(tmp_doc_both)
    return final_similar_desc  

In [35]:
def similarity_df_gen(final_similar_desc):
    final_pd = pd.DataFrame(final_similar_desc,columns=["Rwrd_Description", "Similar_Rwrd_1_desc", 
           "Similar_Rwrd_2_desc", "Similar_Rwrd_3_desc", "Similar_Rwrd_4_desc", "Similar_Rwrd_5_desc", 
           "Rwrd_Name", "Similar_Rwrd_1_name", "Similar_Rwrd_2_name", "Similar_Rwrd_3_name", "Similar_Rwrd_4_name", 
                                                    "Similar_Rwrd_5_name",'score'])
    final_pd = final_pd[["Rwrd_Name", "Rwrd_Description",'score', "Similar_Rwrd_1_name", "Similar_Rwrd_1_desc", 
                         "Similar_Rwrd_2_name", "Similar_Rwrd_2_desc", "Similar_Rwrd_3_name", "Similar_Rwrd_3_desc", 
                         "Similar_Rwrd_4_name", "Similar_Rwrd_4_desc", "Similar_Rwrd_5_name", "Similar_Rwrd_5_desc"]]
    return final_pd

### Calculation Module

#### iteratively generate overall matrice / indice with ranking / argsort included /

In [36]:
% time

overall_indice=[np.zeros((len(details_shopping),len(details_shopping))),
     np.zeros((len(details_beauty),len(details_beauty))),
     np.zeros((len(details_fnb),len(details_fnb)))]
sorted_indice = [[],[],[]]
result_df_lst = [pd.DataFrame(),pd.DataFrame(),pd.DataFrame()]

for idx, dataframe in enumerate([details_shopping, details_beauty, details_fnb]):
    for xxx in ['voucher','desc','tnc']:
        tfidf, tfidf_scores = tfidf_analysis(dataframe, xxx)
        overall_indice[idx] = overall_indice[idx] + words_similarity(tfidf) 
    sorted_indice[idx] = overall_indice[idx].argsort()
    final_similar_desc = []
#     a = similar_reward_extraction(sorted_indice[idx])
#     result_df_lst[idx] = similarity_df_gen(a) 
    result_df_lst[idx] = similarity_df_gen(similar_reward_extraction(sorted_indice[idx]))     

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 11 µs


In [38]:
result_df_lst[0]

,Rwrd_Name,Rwrd_Description,score,Similar_Rwrd_1_name,Similar_Rwrd_1_desc,Similar_Rwrd_2_name,Similar_Rwrd_2_desc,Similar_Rwrd_3_name,Similar_Rwrd_3_desc,Similar_Rwrd_4_name,Similar_Rwrd_4_desc,Similar_Rwrd_5_name,Similar_Rwrd_5_desc
0,rebatemango: up to free rm100 cashback,instantly get up to free rm100 cashback into y...,5,jamaica blue: 20% off,"enjoy 20% off award-winning coffee, mueslis, b...",himalaya: 15% off,15% off on second item\n\n,tealive: free tealive drink worth rm6.50 (wk2),free tealive drink worth rm6.50,dieyalicious: free member registration,free member registration with any purchase at ...,zalora: rm25 off with minimum spend of rm100 (...,rm25 off with minimum spend of rm100
1,maylana: 10% discount for all items,10% discount on all items,5,carlo rino: 10% discount,10% discount on normal priced and 10% discount...,the honest treat: get 30% off,"enjoy 30% off raw, vegan cakes & ice-cream at ...",tomei gold and jewellery: rm100 instant rebate,get additional rm100 instant rebate on top of ...,agoda: additional 8% off in january,additional 8% off on your booking,52nd street saloon: 30% off on konxept product,30% off on konxept product
2,house california: free 30000mah power bank,1. free 30000mah power bank with purchase of r...,5,kitchen shop: up to 50% discount,up to 50% discount on selected brabantia and l...,agoda: additional 10% off in january,additional 10% off on your booking,gnc: free 1 lac fullcal mini or lac taut colla...,redeem 1 (one) lac fullcal mini or lac taut co...,u pizzeria: get 25% off,"enjoy 25% off authentic, freshly baked italian...",anakku: 30% discount on disney new range,30% discount on disney new range including toi...
3,wichday cafe: get 25% off,"enjoy 25% off sandwiches, toasts, pastas & des...",5,baskin-robbins: get free upsize to single king...,free upsize single regular to single king scoo...,gokul yoga surya: free one (1) time trial class,free one (1) time trial class in gokul yoga surya,jari jari spa: 15% discount,15% discount on body spa treatment and couple ...,imax smart outlet: free data usb cable,free data usb cable (android/ apple/type-c) wi...,regus: complimentary phone answering,complimentary phone answering for the first si...
4,big apple: free 1 donut,get 1 free donut (any flavor) when you buy any...,5,eclipse: 10% discount for all items,10% discount on all items,juice works: free upsize on regular prize drinks,get free upsize on regular prize drinks,pierre cardin lingerie: 5% off,5% off on total bill\n,aenon the health kitchen: 25% off,"enjoy 25% off healthy, wholesome meals at aeno...",rotiboy: get free rotiboy signature bun,download mydigi app for one free rotiboy signa...
5,wing tai warehouse sale: additional 10% off,enjoy the preview day with additional 10% off ...,5,eu yan sang: rm10 rebate,rm10 rebate with minimum purchase of rm88 in e...,hush puppies footwear: rm50 gift voucher,rm50 gift voucher with minimum purchase of rm2...,maylana: 10% discount for all items,10% discount on all items,boye café: 20% off,"enjoy 20% off big breakfasts, pastas, baguette...",shins: free celeb beaute celeb nail,free celeb beaute celeb nail (worth rm29.80) w...
6,kitchen shop: up to 50% discount,up to 50% discount on selected brabantia and l...,5,brownies bar: get 20% off,"enjoy 20% off rich, decadent brownies at brown...",suria sabah shopping mall: free digi umbrella,free digi umbrella when you spend minimum rm20...,obermain: rm50 gift voucher,rm50 gift voucher with minimum purchase of rm2...,lazada: 30% off on petron engine oil,30% off and free shipping on petron fully synt...,juice works: free upsize on regular prize drinks,get free upsize on regular prize drinks
7,sunway lagoon: get 32% off admission ticket,get 32% off sunway lagoon admission ticket,4,aeon wellness: buy 1 free 1,buy 1 free 1 (washable place mat)\n,chequers: 25% off,"enjoy 25% off waffles, pizzas & snacks at cheq...",gallery korea décor: rm50 cash voucher,get rm50 cash voucher for purchase over rm500 ...,gokul yoga 

In [39]:
# import heapq
# # heapq.heapify(list(overall_indice[2][0]))
# some = list(overall_indice[2][0])
# heapq.nlargest(3,some)

### Appendix

* Check the first string in VoucherName (made as location according to the function "voucher_name_split") and MerchantName

In [ ]:
def doublecheck(input):
    if input.location == input.MerchantName:
        return 0
    elif input.location in input.MerchantName:
        return 0
    elif input.MerchantName in input.location:
        return 0
    else:
        return 1
    
sum([doublecheck(details.iloc[idx]) for idx in range(details.shape[0])])

* So there are 55 mismatch of location and merchant name. For example,  

In [ ]:
print details.location.iloc[84]
print details.MerchantName.iloc[84]

### Currently under-doubt Features

* For very precise result, we can use remarks to adjust (i.e., + or -) the value to the reward.

In [ ]:
details.Remarks.unique()

* Can i know that 'length' stand for what

In [ ]:
details.Length.unique()

### Other Features

* Feature meaning

In [ ]:
title

In [ ]:
details.iloc[0].voucher